#Semantic Search using SBERT quran data


In [ ]:
!pip install sentence-transformers

In [ ]:
import json

try:
    with open('/content/QURAN_Muyassar.json', 'r') as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")


In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import torch

if not torch.cuda.is_available():
  print("Warning: No GPU detected. Processing will be slow. Please add a GPU to this notebook")

model_name = 'omarelshehy/Arabic-STS-Matryoshka-V2'
#model_name = 'omarelshehy/Arabic-STS-Matryoshka-V2'
model = SentenceTransformer(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:


# Extract attributes and create documents
documents = []
for entry in data:  # Adjust based on the JSON structure (e.g., list of objects)
    if isinstance(entry, dict):  # Ensure each entry is a dictionary
        sura_name = entry.get("sura_name", "")
        #aya_chakl = entry.get("aya_chakl", "")
        aya_no_chakl = entry.get("aya_no_chakl", "")
        aya_index_sura = entry.get("aya_index_sura", "")
        tafsir_aya = entry.get("tafsir_aya", "")

        # Add each attribute as a separate document
        combined_text = f"السورة: {sura_name} \n الاية:|{aya_index_sura}| {aya_no_chakl} \nالتفسير : {tafsir_aya}"
        documents.append(combined_text)




In [ ]:
len(documents)

6235

In [ ]:
documents[0]

'السورة: سورة الفاتحة \n الاية:|1| بسم الله الرحمن الرحيم \nالتفسير : سورة الفاتحة سميت هذه السورة بالفاتحة؛ لأنه يفتتح بها القرآن العظيم، وتسمى المثاني؛ لأنها تقرأ في كل ركعة، ولها أسماء أخر. أبتدئ قراءة القرآن باسم الله مستعينا به، (اللهِ) علم على الرب -تبارك وتعالى- المعبود بحق دون سواه، وهو أخص أسماء الله تعالى، ولا يسمى به غيره سبحانه. (الرَّحْمَنِ) ذي الرحمة العامة الذي وسعت رحمته جميع الخلق، (الرَّحِيمِ) بالمؤمنين، وهما اسمان من أسمائه تعالى، يتضمنان إثبات صفة الرحمة لله تعالى كما يليق بجلاله.'

In [ ]:
import pandas as pd

df = pd.DataFrame(documents,columns=['docs'])

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Example dataset

#df = pd.DataFrame(documents,columns='docs')

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2')
model = AutoModel.from_pretrained('Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2')

def tokenize_and_count(text):
    # Tokenize without truncation to get the full token count
    encoded_full_input = tokenizer(text, padding=False, truncation=False, return_tensors='pt')
    full_tokens = tokenizer.convert_ids_to_tokens(encoded_full_input['input_ids'][0])  # All tokens
    full_token_count = len(full_tokens)

    # Tokenize with truncation to limit the input for processing
    encoded_truncated_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    truncated_tokens = tokenizer.convert_ids_to_tokens(encoded_truncated_input['input_ids'][0])
    truncated_token_count = len(truncated_tokens)

    return full_tokens, full_token_count, truncated_tokens, truncated_token_count


In [ ]:
df['full_tokens'], df['full_token_count'], df['truncated_tokens'], df['truncated_token_count'] = zip(*df['docs'].apply(tokenize_and_count))




In [ ]:
df.to_csv("data_truth.csv")

In [ ]:
filtered_df = df[df['full_token_count']>= 512]


In [ ]:
filtered_df

,docs,full_tokens,full_token_count,truncated_tokens,truncated_token_count
288,السورة: سورة البقرة\nالاية: يا أيها الذين آمنو...,"[[CLS], السور, ##ة, :, سورة, البقرة, الاي, ##ة...",527,"[[CLS], السور, ##ة, :, سورة, البقرة, الاي, ##ة...",512


In [ ]:
# since there is just one document that exceeds the number of tokens(512),
# we will embedd the documents normaly using sentence transformers library
# without chunking

In [ ]:

if documents:
    embeddings = model.encode(documents, show_progress_bar=True, convert_to_tensor=True)
    print(f"Encoded {len(documents)} documents into embeddings.")

In [ ]:
if embeddings !=[]:
    import torch
    torch.save({ "embeddings": embeddings}, 'embeddings.pt')
    print("Embeddings saved to 'embeddings.pt'.")
else:
    print("No documents found to embed.")


Embeddings saved to 'embeddings.pt'.


### Semantic search with Sentence transformers Library


In [ ]:
# Function that searches the corpus and prints the results
def search(inp_question):
    start_time = time.time()
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, embeddings)
    end_time = time.time()
    hits = hits[0]  #Get the hits for the first query

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for i, hit in enumerate(hits[0:5], start=1):
      print(f"Score: {hit['score']:.3f}\nDocument {i}:\n{documents[hit['corpus_id']]}")
      print("--------------------------")




In [ ]:
search("  لمن تعطى الصدقات")

Input question:   لمن تعطى الصدقات
Results (after 0.016 seconds):
Score: 0.620
Document 1:
السورة: سورة التوبة 
 الاية:|60| إنما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضة من الله والله عليم حكيم 
التفسير : إنما تعطى الزكوات الواجبة للمحتاجين الذين لا يملكون شيئًا، وللمساكين الذين لا يملكون كفايتهم، وللسعاة الذين يجمعونها، وللذين تؤلِّفون قلوبهم بها ممن يُرْجَى إسلامه أو قوة إيمانه أو نفعه للمسلمين، أو تدفعون بها شرَّ أحد عن المسلمين، وتعطى في عتق رقاب الأرقاء والمكاتبين، وتعطى للغارمين لإصلاح ذات البين، ولمن أثقلَتْهم الديون في غير فساد ولا تبذير فأعسروا، وللغزاة في سبيل الله، وللمسافر الذي انقطعت به النفقة، هذه القسمة فريضة فرضها الله وقدَّرها. والله عليم بمصالح عباده، حكيم في تدبيره وشرعه.
--------------------------
Score: 0.506
Document 2:
السورة: سورة الإسراء 
 الاية:|26| وآت ذا القربى حقه والمسكين وابن السبيل ولا تبذر تبذيرا 
التفسير : وأحسِنْ إلى كل مَن له صلة قرابة بك، وأعطه حقه من الإحسان والبر، وأعط المسكين ا